# Opracowanie formatu pliku:
plik jest podzielony na trzy części:
1. Informacja o drzewie kompresji
2. Ilość nadmiarowych bitów
3. zakodowane dane

## Drzewo kompresji
Drewo jest przechowywane jako ciąg znaków zakodowanych w UTF-8. Znaki te są zapisem liści oraz operacji sklejania drzew (operacjie oznaczam znakiem "__$__") w odwrotnej notacji polskiej. Korzystam z własności że każde drzewo jest sklejone z dwóch (wynika to z tego jak budujemy drzewo).<br>
Informacja na temat drzewa kompresji kończy się znakiem "__#__"<br>
Jeśli w drzewie pojawi się jakiś znak specjalny ["$", "#", "\"] to będzie on porzedzony znakiem ucieczki "__\\__"

## Ilość nadmiarowych bitów
z racji że plik wypełniany jest bitami tak aby sumaryczna ilość bitów była podzielna przez 8 to występuje problem za durzej ilości danych. w tym celu 4 kolejne bity po informacji o drzewie informują ile będzie bitów dopisanych do końca pliku. takich bitów będzię maksymalnie 7 dlategoteż wykorzystuje tylko 4 bity danych do przechowania tej informacji.

## Zakodowane dane
Właściwe dane zakodowane algorytmem Huffmana na których to końcu są dopisane nadmiarowe bity


In [72]:
from queue import PriorityQueue
import bitstring
from collections import deque

class CodingTreeNode:
    def __init__(self,val=None, left=None, right=None):
        # representation atribute is string that represents object using RPN
        # for example string ab$ represents:
        #               .
        # ab$   -->    / \
        #             a   b
        self.left = left
        self.right = right

        if val == "\\":
            val = "\\\\"
        elif val == "$":
            # $ is marker for compere tree operation
            val = "\\$"
        elif val == "#":
            # # is marker for end of tree in compresed file
            val = "\\#"
        self.val = val
        self.parent = None
        if val is None:
            self.representation = left.representation + right.representation + "$"
        else:
            self.representation = val
    
    def __str__(self) -> str:
        return self.representation
    
    def __getitems(self, headCode):
        if self.left is None:
            return [(self.val, headCode)]
        else:
            return self.left.__getitems(headCode + "0") + self.right.__getitems(headCode + "1")

    def getCodeDict(self):
        codeDict = dict()
        for key, val in self.__getitems(""):
            codeDict[key] = bitstring.Bits(bin=val)
        return codeDict

    def __lt__(self, other):
        return True

In [73]:
def _merge(a, b):
    incidence = a[0] + b[0]
    merged_tree = CodingTreeNode(left = a[1], right = b[1])
    return (incidence, merged_tree)

def static_huffman(data: str, compered_file_name: str) -> None:
    # calculating incidence of letters
    alphabet = dict()
    for letter in data:
        if letter in alphabet:
            alphabet[letter] += 1
        else:
            alphabet[letter] = 1

    # bilding coding tree
    q = PriorityQueue()
    for key, value in alphabet.items():
        q.put((value, CodingTreeNode(val=key)))
    tree = None
    while True:
        a = q.get()
        if q.empty():
            tree = a[1]
            break
        b = q.get()
        q.put(_merge(a,b))

    # coding data
    codeDisct = tree.getCodeDict()
    coded_data = bitstring.BitArray()
    for letter in data:
            coded_data.append(codeDisct[letter])
    overdata_size = (8 - (len(coded_data)+4) % 8) % 8
    coded_data.insert(bitstring.Bits(int=overdata_size, length=4), 0)
    # saving data to file
    with open(compered_file_name, "w", encoding="utf-8") as f:
        f.write(tree.representation + "#")        
    with open(compered_file_name, "ab") as f:
        coded_data.tofile(f)


In [78]:
class BinTree:
    def __init__(self, left, right) -> None:
        self.left = left
        self.right = right

def rebuild_tree(tree_string):
    Nodes = []
    i = 0
    n = len(tree_string)
    while i < n:
        letter = tree_string[i]
        if letter == "\\":
            Nodes.append(tree_string[i:i+2])
            i += 2
        else:
            Nodes.append(letter)
            i += 1
    lowest = len(Nodes) - 1
    def _rebuild_tree(Nodes, i):
        nonlocal lowest
        lowest = min(lowest, i)
        if Nodes[i] == "$":
            right = _rebuild_tree(Nodes, lowest-1)
            left = _rebuild_tree(Nodes, lowest-1)
            return BinTree(left, right)
        elif Nodes[i][0] == "\\":
            return Nodes[i][1:]
        else:
            return Nodes[i]
        
    return _rebuild_tree(Nodes, lowest)


def decode_huffman(file: str) -> str:
    tree_str = ""
    with open(file, "rb") as f:
        # reading tree
        char = f.read(1).decode("utf-8")
        while char != "#":
            tree_str += char
            if char == "\\":
                tree_str += f.read(1).decode("utf-8")
            char = f.read(1).decode("utf-8")
        # reading data
        data = bitstring.BitArray(f.read())
        
    # decoding data
    root = rebuild_tree(tree_str)
    wanderer = root
    decoded_text = ""
    overdata_size = data[:4].int
    for bit in data[4:-overdata_size+1]:
        if not isinstance(wanderer, BinTree):
            decoded_text += wanderer
            wanderer = root
        if bit:
            wanderer = wanderer.right
        else:
            wanderer = wanderer.left
    return decoded_text

    

In [85]:
with open("some_file(1)", "r") as f:
    text = f.read()
static_huffman(text, "some_file")

text2 = decode_huffman("some_file")

if text == text2:
    print("hurra")

hurra
